In [100]:
import os
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
train_dataset= pd.read_csv('C:\\Users\\admin\\Documents\\Python_Scripts\\NP_Binary_COE_contest\\ds-coe\\Training_Dataset.csv')
test_dataset = pd.read_csv('C:\\Users\\admin\\Documents\\Python_Scripts\\NP_Binary_COE_contest\\ds-coe\\Test_Dataset.csv')
sample_submission = pd.read_csv('C:\\Users\\admin\\Documents\\Python_Scripts\\NP_Binary_COE_contest\\ds-coe\\Sample_Submission.csv')

In [4]:
pd.set_option('display.max_colwidth', None)

In [19]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#lemma test
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk import bigrams
from nltk.util import ngrams

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
#Understanding the dataset
input_df = train_dataset.copy()
input_df['len_of_text'] = input_df['text'].astype(str).map(len)
input_df

,ID,text,label,len_of_text
0,18822638,"I should have known better, its obviously not a juul. I was just happy to see Oliver Tree tbh",0,94
1,9f9f1c3f,"I am happy for you. No, seriously, I am.",0,40
2,563242ec,Everyone including an ambassador has freedom of speech. Get an education.,0,73
3,ceda51ca,Oh really?,0,10
4,bfb0c83d,"Thanks for reply, I'll enjoy reading through the comments... maybe you make a book of it all.",0,94
...,...,...,...,...
24304,2bf61e0d,"Suffocation, no breathing",0,25
24305,6cadee8d,They both do! Like wow!!!,0,25
24306,57eaa4da,"I also posted about this a few days ago. Maybe they have different designers but none of them has follow-up or approval from a single person, unfortunate",1,153
24307,f1c2acd8,"Look into BRI countries, pretty strong overlap with them would be my guess.",0,76


In [18]:
input_df.info() #no null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24309 entries, 0 to 24308
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           24309 non-null  object
 1   text         24309 non-null  object
 2   label        24309 non-null  int64 
 3   len_of_text  24309 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 759.8+ KB


In [19]:
input_df.describe() #percentile distribuion of values

,label,len_of_text
count,24309.000000,24309.000000
mean,0.210580,69.697561
std,0.407729,37.455002
min,0.000000,2.000000
25%,0.000000,39.000000
50%,0.000000,66.000000
75%,0.000000,97.000000
max,1.000000,703.000000


In [20]:
input_df.corr() #as expected no relation to the len of text and the sentiment

,label,len_of_text
label,1.000000,0.042918
len_of_text,0.042918,1.000000


In [85]:
input_df.count().head(1), input_df[input_df['label']==1].count().head(1), input_df[input_df['label']==0].count().head(1)
#4:1 class imbalance

(ID    24309
 dtype: int64,
 ID    5119
 dtype: int64,
 ID    19190
 dtype: int64)

In [79]:

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatize Single Word
print(lemmatizer.lemmatize("making"))
print(lemmatizer.lemmatize("make"))
print(lemmatizer.lemmatize("mak"))

making
make
mak


In [50]:
#split into test and train
train_cp_df, test_cp_df = train_test_split(train_dataset,test_size = 0.1)
train_df1 = train_cp_df.copy()
test_df = test_cp_df.copy()

In [51]:
train_df1.count().head(1), train_df1[train_df1['label']==1].count().head(1), train_df1[train_df1['label']==0].count().head(1)

(ID    21878
 dtype: int64,
 ID    4585
 dtype: int64,
 ID    17293
 dtype: int64)

In [254]:
test_df.count().head(1), test_df[input_df['label']==1].count().head(1), test_df[input_df['label']==0].count().head(1)
#similar split

<ipython-input-254-f260f19e6257>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df.count().head(1), test_df[input_df['label']==1].count().head(1), test_df[input_df['label']==0].count().head(1)
<ipython-input-254-f260f19e6257>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df.count().head(1), test_df[input_df['label']==1].count().head(1), test_df[input_df['label']==0].count().head(1)


(ID    2431
 dtype: int64,
 ID    534
 dtype: int64,
 ID    1897
 dtype: int64)

In [81]:
train_df = train_df1.copy() #no oversampling

In [70]:
#oversampling - twice
lbl_df = train_df1[train_df1['label']==1]
lbl1_df = lbl_df.copy()
lbl1_df['ID'] = lbl1_df['ID']+'xx1'
lbl2_df = lbl_df.copy()
lbl2_df['ID'] = lbl2_df['ID']+'xx2'
train_df2 = pd.concat([train_df1,lbl1_df])
train_df = pd.concat([train_df2,lbl2_df])
train_df.count().head(1), train_df[train_df['label']==1].count().head(1), train_df[train_df['label']==0].count().head(1)

(ID    31048
 dtype: int64,
 ID    13755
 dtype: int64,
 ID    17293
 dtype: int64)

In [82]:
#remove stop words

stop = stopwords.words('english')
train_df['text_stop'] = train_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [61]:
#word tokenize
#train_df['text_wtkn_stop'] = train_df['text'].apply(lambda x: word_tokenize(x))
#train_df['text_stkn_stop'] = train_df['text'].apply(lambda x: sent_tokenize(x))
#train_df['text_stkn_stop'] = train_df['text'].apply(sent_tokenize).apply(lambda x: " ".join(x for x in x.split() if x not in stop))

<ipython-input-61-035fe6f80936>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text_wtkn_stop'] = train_df['text'].apply(lambda x: word_tokenize(x))
<ipython-input-61-035fe6f80936>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text_stkn_stop'] = train_df['text'].apply(lambda x: sent_tokenize(x))


In [113]:
train_df

,ID,text,label,text_stop
3868,b8feec0e,I couldn't let Christina Payne become carver.....,1,I let Christina Payne become carver.. I regret...
8707,4e0109d4,Why would the landlord pay it if electric isnt...,1,Why would landlord pay electric isnt included....
10891,4d84fbd6,"You’re right, 5 minutes won’t be enough, earli...",0,"You’re right, 5 minutes won’t enough, earlier ..."
15531,42a63924,...and his last one was in New Orleans. Let’s ...,0,...and last one New Orleans. Let’s go Rams!
5526,f5878f9c,> mined the worldwide largest block It was orp...,0,"> mined worldwide largest block It orphaned, u..."
...,...,...,...,...
13475,1a875d4b,Canadian Kristi Wolfe here to disagree with yo...,1,Canadian Kristi Wolfe disagree you. Politely. ...
15110,03eb2fc2,It's true though. He either gets no shirt and ...,0,It's true though. He either gets shirt freezes...
21388,2dd3be45,Wow he is very gifted! I hope he continues to ...,0,Wow gifted! I hope continues freeze buffalos!
23622,cf45303a,"Keep them until November, and then return them.",0,"Keep November, return them."


In [182]:
train_df.iloc[3239]

ID                                                                                                                          053c9566
text          But perhaps it opens a door to your escape from a lifetime of dealing with that. I would call it a fortuitous misstep.
label                                                                                                                              0
text_stop                                      But perhaps opens door escape lifetime dealing that. I would call fortuitous misstep.
emoji_text                                     But perhaps opens door escape lifetime dealing that. I would call fortuitous misstep.
lemma_text                                      But perhaps open door escape lifetime dealing that. I would call fortuitous misstep.
Name: 17155, dtype: object

In [83]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
train_df['emoji_text'] = train_df['text_stop'].apply(lambda x: remove_emoji(x))

In [124]:
train_df

,ID,text,label,text_stop,emoji_text,lemma_text
13423,bcec4c5a,Don't forget the price increase. They made sur...,0,Don't forget price increase. They made sure ke...,Don't forget price increase. They made sure ke...,"[Don't, forget, price, increase., They, made, ..."
2841,86995857,Hahahahahaha you 100 percent got my ass first ...,0,Hahahahahaha 100 percent got ass first time mo...,Hahahahahaha 100 percent got ass first time mo...,"[Hahahahahaha, 100, percent, got, as, first, t..."
22402,ecdfbc63,The saddest part is that just this morning whe...,0,The saddest part morning laying bed I thinking...,The saddest part morning laying bed I thinking...,"[The, saddest, part, morning, laying, bed, I, ..."
24146,72bcf333,If your walls are not connected sprinkle your ...,0,If walls connected sprinkle lawn regularly dia...,If walls connected sprinkle lawn regularly dia...,"[If, wall, connected, sprinkle, lawn, regularl..."
7697,890282df,Except they miss most of those too.,0,Except miss too.,Except miss too.,"[Except, miss, too.]"
...,...,...,...,...,...,...
23287,976acd7e,Luckily I don’t like beer so I don’t have to w...,0,Luckily I don’t like beer I don’t worry that.....,Luckily I don’t like beer I don’t worry that.....,"[Luckily, I, don’t, like, beer, I, don’t, worr..."
14740,df383fc4,My girlfriend ordered Chipotle. Have fun with ...,0,My girlfriend ordered Chipotle. Have fun 15 mi...,My girlfriend ordered Chipotle. Have fun 15 mi...,"[My, girlfriend, ordered, Chipotle., Have, fun..."
1818,f3fbc8d0,What exactly is the point of stating something...,0,What exactly point stating something “right al...,What exactly point stating something “right al...,"[What, exactly, point, stating, something, “ri..."
21902,9fede820,Nah I think she probably knows he will side wi...,0,Nah I think probably knows side mom lol,Nah I think probably knows side mom lol,"[Nah, I, think, probably, know, side, mom, lol]"


In [84]:
#lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    lemma_words = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return ' '.join(lemma_words)


train_df['lemma_text'] = train_df['emoji_text'].apply(lemmatize_text)
train_df.head()

,ID,text,label,text_stop,emoji_text,lemma_text
14816,17b74dfe,"Now that's brilliant... 30? woah. I'm only 19, hope it doesn't take me that long",0,"Now that's brilliant... 30? woah. I'm 19, hope take long","Now that's brilliant... 30? woah. I'm 19, hope take long","Now that's brilliant... 30? woah. I'm 19, hope take long"
19366,4078234f,"Boondock Saints, a brilliant documentary on the South Boston psyche",0,"Boondock Saints, brilliant documentary South Boston psyche","Boondock Saints, brilliant documentary South Boston psyche","Boondock Saints, brilliant documentary South Boston psyche"
13603,4192b4a5,Then enjoy your shutdown!,0,Then enjoy shutdown!,Then enjoy shutdown!,Then enjoy shutdown!
20320,2bd5cbc5,"No, I prefer having a forest",1,"No, I prefer forest","No, I prefer forest","No, I prefer forest"
965,c77e2b23,Came here to say this. I thank you kindly. Long live America. 😁,0,Came say this. I thank kindly. Long live America. 😁,Came say this. I thank kindly. Long live America.,Came say this. I thank kindly. Long live America.


In [74]:
#stem
stemmer = SnowballStemmer("english")
def stemming_text(text):
    stem_words = [stemmer.stem(w) for w in w_tokenizer.tokenize(text)]
    return ' '.join(stem_words)

train_df['stem_text'] = train_df['lemma_text'].apply(stemming_text)

In [214]:
train_df

,ID,text,label,text_stop,emoji_text,lemma_text,stem_text
13423,bcec4c5a,Don't forget the price increase. They made sure to keep that promise.,0,Don't forget price increase. They made sure keep promise.,Don't forget the price increase. They made sure to keep that promise.,Don't forget the price increase. They made sure to keep that promise.,don't forget the price increase. they made sure to keep that promise.
2841,86995857,Hahahahahaha you 100 percent got my ass first time in months I’ve been got. Thank you.,0,Hahahahahaha 100 percent got ass first time months I’ve got. Thank you.,Hahahahahaha you 100 percent got my ass first time in months I’ve been got. Thank you.,Hahahahahaha you 100 percent got my as first time in month I’ve been got. Thank you.,hahahahahaha you 100 percent got my ass first time in month i'v been got. thank you.
22402,ecdfbc63,The saddest part is that just this morning when we were laying in bed I was thinking about how much I love him.,0,The saddest part morning laying bed I thinking much I love him.,The saddest part is that just this morning when we were laying in bed I was thinking about how much I love him.,The saddest part is that just this morning when we were laying in bed I wa thinking about how much I love him.,the saddest part is that just this morn when we were lay in bed i was think about how much i love him.
24146,72bcf333,If your walls are not connected sprinkle your lawn in between regularly with diatomaceous earth. It will kill them before they can get to you.,0,If walls connected sprinkle lawn regularly diatomaceous earth. It kill get you.,If your walls are not connected sprinkle your lawn in between regularly with diatomaceous earth. It will kill them before they can get to you.,If your wall are not connected sprinkle your lawn in between regularly with diatomaceous earth. It will kill them before they can get to you.,if your wall are not connect sprinkl your lawn in between regular with diatomac earth. it will kill them befor they can get to you.
7697,890282df,Except they miss most of those too.,0,Except miss too.,Except they miss most of those too.,Except they miss most of those too.,except they miss most of those too.
...,...,...,...,...,...,...,...
23287,976acd7e,Luckily I don’t like beer so I don’t have to worry about that.. but my love for bread more than makes up for that!,0,Luckily I don’t like beer I don’t worry that.. love bread makes that!,Luckily I don’t like beer so I don’t have to worry about that.. but my love for bread more than makes up for that!,Luckily I don’t like beer so I don’t have to worry about that.. but my love for bread more than make up for that!,luckili i don't like beer so i don't have to worri about that.. but my love for bread more than make up for that!
14740,df383fc4,My girlfriend ordered Chipotle. Have fun with that 15 mile drive bud.,0,My girlfriend ordered Chipotle. Have fun 15 mile drive bud.,My girlfriend ordered Chipotle. Have fun with that 15 mile drive bud.,My girlfriend ordered Chipotle. Have fun with that 15 mile drive bud.,my girlfriend order chipotle. have fun with that 15 mile drive bud.
1818,f3fbc8d0,What exactly is the point of stating something as being a “right aligned opinion” then?,0,What exactly point stating something “right aligned opinion” then?,What exactly is the point of stating something as being a “right aligned opinion” then?,What exactly is the point of stating something a being a “right aligned opinion” then?,what exact is the point of state someth as be a “right align opinion” then?
21902,9fede820,Nah I think she probably knows he will side with her mom lol,0,Nah I think probably knows side mom lol,Nah I think she probably knows he will side with her mom lol,Nah I think she probably know he will side with her mom lol,nah i think she probabl know he will side with her mom lol


In [85]:

#Count vectorizer model
count_vec = CountVectorizer(ngram_range=(1,3), max_features=2000, strip_accents='unicode')
count_vec.fit(train_df['lemma_text'])
train_vectorized = count_vec.transform(train_df['lemma_text'])


In [86]:
rnd_mdl = RandomForestClassifier()
rnd_mdl.fit(train_vectorized, train_df['label'])
#train accuracy
y_train_pred = rnd_mdl.predict(train_vectorized)

In [290]:
rnd_mdl

RandomForestClassifier()

In [87]:
train_f1 = f1_score(train_df['label'], y_train_pred)
train_f1

0.9855488141202426

In [88]:
#internal test data check
test_df['text_stop'] = test_df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_df['emoji_text'] = test_df['text_stop'].apply(lambda x: remove_emoji(x))
test_df['lemma_text'] = test_df['emoji_text'].apply(lemmatize_text)
#test_df['stem_text'] = test_df['text'].apply(stemming_text)
test_vectorized = count_vec.transform(test_df['lemma_text'])
y_test_pred = rnd_mdl.predict(test_vectorized)

In [89]:
test_f1 = f1_score(test_df['label'], y_test_pred)
test_f1

0.4649610678531702

In [90]:
#no oversampling
print(confusion_matrix(test_df['label'], y_test_pred), accuracy_score(test_df['label'], y_test_pred),
precision_score(test_df['label'], y_test_pred), recall_score(test_df['label'], y_test_pred),
f1_score(test_df['label'], y_test_pred), f1_score(test_df['label'], y_test_pred,'weighted'))

[[1741  156]
 [ 325  209]] 0.8021390374331551 0.5726027397260274 0.3913857677902622 0.4649610678531702 0.4649610678531702


C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=weighted as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [80]:
#twice oversampling
print(confusion_matrix(test_df['label'], y_test_pred), accuracy_score(test_df['label'], y_test_pred),
precision_score(test_df['label'], y_test_pred), recall_score(test_df['label'], y_test_pred),
f1_score(test_df['label'], y_test_pred), f1_score(test_df['label'], y_test_pred,'weighted'))

[[1645  252]
 [ 276  258]] 0.7828054298642534 0.5058823529411764 0.48314606741573035 0.49425287356321834 0.49425287356321834


C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=weighted as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [67]:
#twice oversampling
print(confusion_matrix(test_df['label'], y_test_pred), accuracy_score(test_df['label'], y_test_pred),
precision_score(test_df['label'], y_test_pred), recall_score(test_df['label'], y_test_pred),
f1_score(test_df['label'], y_test_pred), f1_score(test_df['label'], y_test_pred,'weighted'))

[[1674  223]
 [ 286  248]] 0.79062114356232 0.5265392781316348 0.46441947565543074 0.4935323383084577 0.4935323383084577


C:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=weighted as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [287]:
rnd_mdl.feature_importances_

array([0.00017625, 0.00046396, 0.00025634, ..., 0.00015675, 0.00014385,
       0.00037639])

In [220]:
y_test_pred[:20], test_df['label'][:20]

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       dtype=int64),
 22948    0
 7349     0
 9133     0
 16038    0
 6131     1
 4547     0
 17163    0
 16474    0
 10028    1
 3239     1
 6111     0
 13784    0
 8081     0
 15320    0
 22467    1
 4227     0
 2761     0
 866      0
 11594    0
 6013     0
 Name: label, dtype: int64)

In [279]:
sample_submission.head()

,ID,label
0,284f775a,0
1,1dd04583,0
2,49e1e335,0
3,8b86a110,0
4,59f206bf,0


In [91]:

#external test data check
test_dataset['text_stop'] = test_dataset['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_dataset['emoji_text'] = test_dataset['text_stop'].apply(lambda x: remove_emoji(x))
test_dataset['lemma_text'] = test_dataset['emoji_text'].apply(lemmatize_text)
#test_df['stem_text'] = test_df['text'].apply(stemming_text)
test_dataset_vectorized = count_vec.transform(test_dataset['lemma_text'])
y_final_pred = rnd_mdl.predict(test_dataset_vectorized)
sample_submission['label'] = y_final_pred

In [92]:
sample_submission['label'].value_counts(normalize=True)*100

0    84.3747
1    15.6253
Name: label, dtype: float64

In [109]:
sample_submission.to_csv("rnd1_ngram_stop_lemma__2oversampl_submission.csv", index=False)

In [285]:
#Naive Bayes
nb_mdl = MultinomialNB()
nb_mdl.fit(train_vectorized, train_df['label'])
#train accuracy
y_train_pred = rnd_mdl.predict(train_vectorized)
train_f1 = f1_score(train_df['label'], y_train_pred)
train_f1

0.9855615562658436

In [286]:
test_vectorized = count_vec.transform(test_df['lemma_text'])
y_test_pred = rnd_mdl.predict(test_vectorized)
print(confusion_matrix(test_df['label'], y_test_pred), accuracy_score(test_df['label'], y_test_pred),
precision_score(test_df['label'], y_test_pred), recall_score(test_df['label'], y_test_pred),
f1_score(test_df['label'], y_test_pred))

[[1726  171]
 [ 307  227]] 0.8033730974907446 0.5703517587939698 0.4250936329588015 0.4871244635193133


In [26]:
#Random grid + CV + random forest
random_grid = {'bootstrap': [True,False],
 'max_depth': [10, 20, 30, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600]}

rf_random = RandomizedSearchCV(estimator = rnd_mdl, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(train_vectorized, train_df['label'])


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 16.3min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600]},
                   random_state=42, verbose=2)

In [27]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': True}

In [30]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [40,80,None],
    'max_features': ['auto'],
    'min_samples_leaf': [2,3],
    'min_samples_split': [2,3,4],
    'n_estimators': [600, 800, 1000]
}
grid_search = GridSearchCV(estimator = rnd_mdl, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(train_vectorized, train_df['label'])

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed: 21.6min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [40, 80, None],
                         'max_features': ['auto'], 'min_samples_leaf': [2, 3],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [600, 800, 1000]},
             verbose=2)

In [33]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 800}

In [45]:
#hypertuned random forest
rnd_mdl_tuned = RandomForestClassifier(max_features='auto', max_depth=None,
                                      min_samples_leaf=2, min_samples_split=2, n_estimators=800, random_state=42)
rnd_mdl_tuned.fit(train_vectorized, train_df['label'])
#train accuracy
y_train_pred = rnd_mdl_tuned.predict(train_vectorized)
train_f1 = f1_score(train_df['label'], y_train_pred)
train_f1

0.5424354243542435

In [46]:
#test_vectorized = count_vec.transform(test_df['lemma_text'])
y_test_pred = rnd_mdl_tuned.predict(test_vectorized)
print(confusion_matrix(test_df['label'], y_test_pred), accuracy_score(test_df['label'], y_test_pred),
precision_score(test_df['label'], y_test_pred), recall_score(test_df['label'], y_test_pred),
f1_score(test_df['label'], y_test_pred))

[[1838   72]
 [ 362  159]] 0.8214726450020567 0.6883116883116883 0.30518234165067176 0.42287234042553196


In [101]:
#repeat plain random forest with tf-idf 
#Count vectorizer model
tfIdfVectorizer=TfidfVectorizer(use_idf=True)
tfIdfVectorizer.fit(train_df['lemma_text'])
train_tdidf = tfIdfVectorizer.transform(train_df['lemma_text'])

In [102]:
rnd_mdl2 = RandomForestClassifier()
rnd_mdl2.fit(train_tdidf, train_df['label'])
#train accuracy
y_train_pred = rnd_mdl2.predict(train_tdidf)
train_f1 = f1_score(train_df['label'], y_train_pred)
train_f1

0.9980366492146596

In [103]:
test_tdidf = tfIdfVectorizer.transform(test_df['lemma_text'])
y_test_pred = rnd_mdl2.predict(test_tdidf)

In [104]:
print(confusion_matrix(test_df['label'], y_test_pred), accuracy_score(test_df['label'], y_test_pred),
precision_score(test_df['label'], y_test_pred), recall_score(test_df['label'], y_test_pred),
f1_score(test_df['label'], y_test_pred))

[[1808   89]
 [ 345  189]] 0.8214726450020567 0.6798561151079137 0.3539325842696629 0.4655172413793103


In [106]:
#external test data check
test_dataset['text_stop'] = test_dataset['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_dataset['emoji_text'] = test_dataset['text_stop'].apply(lambda x: remove_emoji(x))
test_dataset['lemma_text'] = test_dataset['emoji_text'].apply(lemmatize_text)
#test_df['stem_text'] = test_df['text'].apply(stemming_text)
test_dataset_tfidf = tfIdfVectorizer.transform(test_dataset['lemma_text'])
y_final_pred = rnd_mdl2.predict(test_dataset_tfidf)
sample_submission['label'] = y_final_pred

In [107]:
sample_submission['label'].value_counts(normalize=True)*100

0    88.031481
1    11.968519
Name: label, dtype: float64

In [110]:
sample_submission.to_csv("rnd1_ngram_stop_lemma_tf_2oversampl_submission.csv", index=False)